#### 📌 Setup do ambiente e criação da SparkSession

In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("EDA - Rossmann").getOrCreate()

#### 📌 Carregando os dados

In [10]:
train_path = "../data/raw/rossmann_store_sales/train.csv"
store_path = "../data/raw/rossmann_store_sales/store.csv"

df_train = spark.read.option("header", True).option("inferSchema", True).csv(train_path)
df_store = spark.read.option("header", True).option("inferSchema", True).csv(store_path)

#### 📌 Análise inicial dos dados

In [11]:
# Visualizar schema
df_train.printSchema()
df_store.printSchema()

# Visualizar algumas linhas
df_train.show(5)
df_store.show(5)

# Visualizar quantidade de linhas em cada DataFrame
print(f"Train rows: {df_train.count()} | Store rows: {df_store.count()}")

root
 |-- Store: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Customers: integer (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)

root
 |-- Store: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- CompetitionOpenSinceMonth: integer (nullable = true)
 |-- CompetitionOpenSinceYear: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)

+-----+---------+----------+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|      Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoli

#### 📌 Verificando valores nulos

In [18]:
from pyspark.sql.functions import col, count, isnan, when, lit
from pyspark.sql.types import DoubleType, FloatType

def missing_values(df):
    return df.select([
        count(when(col(c).isNull() | (isnan(col(c)) if df.dtypes in ["double", "float"] else lit(False)), c)).alias(c)
        for c in df.columns
    ])

print("Valores nulos/NaN de train")
missing_values(df_train).show()

print("Valores nulos/NaN de store")
missing_values(df_store).show()

Valores nulos/NaN de train


+-----+---------+----+-----+---------+----+-----+------------+-------------+
|Store|DayOfWeek|Date|Sales|Customers|Open|Promo|StateHoliday|SchoolHoliday|
+-----+---------+----+-----+---------+----+-----+------------+-------------+
|    0|        0|   0|    0|        0|   0|    0|           0|            0|
+-----+---------+----+-----+---------+----+-----+------------+-------------+

Valores nulos/NaN de store
+-----+---------+----------+-------------------+-------------------------+------------------------+------+---------------+---------------+-------------+
|Store|StoreType|Assortment|CompetitionDistance|CompetitionOpenSinceMonth|CompetitionOpenSinceYear|Promo2|Promo2SinceWeek|Promo2SinceYear|PromoInterval|
+-----+---------+----------+-------------------+-------------------------+------------------------+------+---------------+---------------+-------------+
|    0|        0|         0|                  3|                      354|                     354|     0|            544|      

#### 📌 Estatísticas descritivas

In [ ]:
df_train.describe(["Sales", "Customers"]).show()

+-------+------------------+-----------------+
|summary|             Sales|        Customers|
+-------+------------------+-----------------+
|  count|           1017209|          1017209|
|   mean| 5773.818972305593|633.1459464082602|
| stddev|3849.9261752347525|464.4117338866323|
|    min|                 0|                0|
|    max|             41551|             7388|
+-------+------------------+-----------------+



#### 📌 Algumas análises específicas 

In [22]:
from pyspark.sql.functions import to_date
# Média de vendas por dia da semana
df_train.groupBy("DayOfWeek").avg("Sales").orderBy("DayOfWeek").show()

# Impacto das vendas
df_train.groupBy("Promo").avg("Sales").show()

# Converter a data para timestamp (para o caso de usos futuros)
df_train_no_timestamp = df_train
df_train = df_train.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

+---------+------------------+
|DayOfWeek|        avg(Sales)|
+---------+------------------+
|        1| 7809.044510467767|
|        2| 7005.244466717926|
|        3| 6555.884138262451|
|        4|  6247.57591278412|
|        5|  6723.27430491275|
|        6| 5847.562599322877|
|        7|204.18318938713466|
+---------+------------------+

+-----+-----------------+
|Promo|       avg(Sales)|
+-----+-----------------+
|    1|7991.152045969903|
|    0|4406.050805160786|
+-----+-----------------+



#### 📌 Merge com os dados de loja

In [25]:
df_merged = df_train.join(df_store, on="Store", how="left")
df_merged.printSchema()
df_merged.select("StoreType", "Sales").groupBy("StoreType").avg("Sales").show(3)

root
 |-- Store: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Sales: integer (nullable = true)
 |-- Customers: integer (nullable = true)
 |-- Open: integer (nullable = true)
 |-- Promo: integer (nullable = true)
 |-- StateHoliday: string (nullable = true)
 |-- SchoolHoliday: integer (nullable = true)
 |-- StoreType: string (nullable = true)
 |-- Assortment: string (nullable = true)
 |-- CompetitionDistance: integer (nullable = true)
 |-- CompetitionOpenSinceMonth: integer (nullable = true)
 |-- CompetitionOpenSinceYear: integer (nullable = true)
 |-- Promo2: integer (nullable = true)
 |-- Promo2SinceWeek: integer (nullable = true)
 |-- Promo2SinceYear: integer (nullable = true)
 |-- PromoInterval: string (nullable = true)

+---------+------------------+
|StoreType|        avg(Sales)|
+---------+------------------+
|        d| 5641.819243109884|
|        c|5723.6292458345515|
|        b|10058.837334175616|
+---------+--------